# CI Portfolio Project 5 - Filter Maintenance Predictor 2022
## **Data Collection Notebook**

## Objectives


* Fetch data from Kaggle and save it as raw data.
* Inspect the data and save it under outputs/datasets/collection

### Inputs

*   Kaggle JSON file - the authentication token.

### Outputs

* Generates Two Datasets: 
    1. outputs/datasets/collection/**PredictiveMaintenanceTest**.csv
    2. outputs/datasets/collection/**PredictiveMaintenanceTrain**.csv

### Additional Comments
* The data is from a publicly accessible Kaggle repo found [here](https://www.kaggle.com/datasets/prognosticshse/preventive-to-predicitve-maintenance) and comes pre-divided into distinctly different Testing and Training data.
* For the purposes of the learning context of this project, we are hosting the data in a publicly accessible repo at [GitHub](https://github.com/roeszler/filter-maintenance-predictor).
* In the workplace, we would never push data to a public repository due to security exposure it represents.

---

# Change working directory

The notebooks are stored in a subfolder. When running the notebook in the editor, we change the working directory from its current folder to its parent folder.
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

To make the parent of the current directory the new current directory
* `os.path.dirname()` = gets the parent directory
* `os.chir()` = defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("Current directory set to new location")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Fetch data from Kaggle

Install an instance of Kaggle to work within the editor

`pip install kaggle==1.5.12`

<!-- We have pre-installed Kaggle package to fetch data using : 

`pip install kaggle==1.5.12` -->

This can be pre included in the requirements.txt documentation to load on initialization using : 

`pip3 freeze --local > requirements.txt`

#### 1. Download a .JSON file (authentication token) from Kaggle and include it in the root directory
* kaggle.json

#### 2. Recognize the token in the session

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

#### 3. Define the Kaggle dataset, and destination folder and download it.

Kaggle url: [/prognosticshse/preventive-to-predicitve-maintenance](https://www.kaggle.com/datasets/prognosticshse/preventive-to-predicitve-maintenance) .
* **Note** the misspelling of 'predictive'

The following function: 
* Retrieves and defines the Kaggle dataset
* Creates a destination folder folder for the data to be placed
* Downloads it to the destination folder
* Unzips the downloaded file
* Deletes the **.zip** file and unused data
* Removes any **kaggle.json** files used to access the dataset on Kaggle

In [ ]:
KaggleDatasetPath = 'prognosticshse/preventive-to-predicitve-maintenance'
DestinationFolder = 'inputs/datasets/raw'   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

! unzip {DestinationFolder}/*.zip -d {DestinationFolder} \
  && rm {DestinationFolder}/*.zip \
  && rm {DestinationFolder}/*.pdf \
  && rm {DestinationFolder}/*.mat \
#   && rm kaggle.json

---

# Load and Inspect Kaggle data

#### Load Data to Inspect
We could combine both datasets, however as they have been included as two sets with slightly different content, we will inspect them each separately.

In [ ]:
import pandas as pd
df_test = pd.read_csv(f'inputs/datasets/raw/Test_Data_CSV.csv')
df_train = pd.read_csv(f'inputs/datasets/raw/Train_Data_CSV.csv')

#### DataFrame Summary

In [ ]:
df_test.info()

In [ ]:
df_train.info()

---

# Explore Data


Pre installed `pandas_profiling` and `ipywidgets` with: 

* `pip install pandas-profiling`

* `pip install ipywidgets`

Not forgetting to update the requirements.txt

Consider the following frameworks to visualize and review the data with:

```
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
```

#### To explore the **Test** dataset:

In [ ]:
df_test.head()

In [ ]:
from pandas_profiling import ProfileReport
pandas_report_test = ProfileReport(df=df_test, minimal=True)
pandas_report_test.to_notebook_iframe()

### Main observations of the **Test** Dataset :

* There are no missing cells.

* Differential Pressure has zero's and has a **reverse exponential** shaped distribution 
    * This correlates to what we understand. The beginning of each test set will have a period where the filter is clean and the difference in pressure is negligible.
    * Subsequently, the measures of distribution (Mean, Median, Mode, Skewness, Kurtosis) correlate to the reverse exponential shape

* Most of the **Dust_Feed** was run at 60mm<sup>3</sup>/s
    * possibly manipulate data to make the range of test sets more evenly distributed

* There is more than three times the amount of A3 Medium Dust observations (47.9%) as there is A2 Fine dust (14.8%), with A4 Course tests (37.3%)
    * possibly manipulate data to make the range of test sets more evenly distributed
    
* The RUL target distribution is right or **positively skewed** at 0.71.
    * Confirmed by the **Mean** of **111.48** > **Median** of **93.5**
    * An ideal normal distribution has mean, median and mode similar in value and a skewness measure approaching zero
    * A measure of the distributions tails; Kurtosis at -0.34 is relatively low in value and negative, indicating few outliers.
    * Similar to **differential pressure** This shape is what we expect for a variable that progresses to zero.

### Early Conclusions
* Further box plot visualization to further investigate this skewness.
* We will consider manipulating data at the feature engineering stage to reduce the affect of skewness, like:
    * Random Forest Selection (Bagging)
    * Logarithmic transformation
    * Manipulate the data range to that of test sets more evenly distributed
    * Feature Scaling

#### Note: 
This dataset has deliberately had the tails of its observations removed at random points (right censored). This needs to be considered when looking at engineering the distributions. In light of this, depending on our Principal Component Analysis (PCA) a Random Forest Selection (Bagging) may present itself as the preferred method to engineer this set.

---

#### To explore the **Train** dataset:

In [ ]:
df_train.head()

In [ ]:
pandas_report_train = ProfileReport(df=df_train, minimal=True)
pandas_report_train.to_notebook_iframe()

What group do the zeros appear in mostly?

### Main observations of the **Train** Dataset :

* There are also no missing cells.

* Differential Pressure has zero's and has the same **reverse exponential** shaped distribution as df_test.
    * This correlates to what we understand. The beginning of each test set will have a period where the filter is clean and the difference in pressure is negligible.
    * Subsequently, the measures of distribution (Mean, Median, Mode, Skewness, Kurtosis) correlate to the same reverse exponential shape

* Most of the **Dust_Feed** was ab bit more evenly spread through the data, a 27% from 158.5mm<sup>3</sup>/s to around 20% in feeds between 60mm<sup>3</sup>/s to 118mm<sup>3</sup>/s.
    * In a live project, we would check the stakeholders as to possible reasons for this and confirm that it represents typical data seen in practice
    * possibly manipulate data to make the range of test sets more evenly distributed

* The dust observations maintain A3 Medium Dust as the highest proportion (47.9%), however the portions of A2 Fine dust (28.2%) to A4 Course Dust (23.7%) are approximately the same.
    * We would also check this with the stakeholders in a live workplace project
    * possibly manipulate data to make the range of test sets more evenly distributed

#### Reminder Note: 
This dataset has deliberately had the tails of its observations removed at random points (right censored). This needs to be considered when looking at engineering the distributions of this dataset. In light of this and further Principal Component Analysis (PCA) a Random Forest Selection (Bagging) may present itself as the preferred method to engineer this set.

---

## Considerations & Manipulation

#### We note that the dataset has **no missing data**. 
* This is outside of what we already know to be true for **df_test** (with RUL) and **df_train** (without RUL).

### Extend and convert `Data_No` of **df_test** dataset

A comparison between sets reveals that the **Data_No** variable:
* Is a categorical variable presented as an integer
* Restarts at the beginning of each dataset

This has the potential to confound subsequent analysis between the sets, where the analysis erroneously considers *Data_No* a discrete value &/or a duplicate entry. To help avoid confusion we alter the values in the **df_test dataset** to be a continuation from the bins seen in the **df_train dataset**.

This is as simple as adding the total number of unique test bins in the df_test set to each one seen in the df_train set:

Quick reminder of the tables we are working with

In [ ]:
df_test.head()

In [ ]:
df_train.head()

Calculate the total number of test sets in **df_train**

In [ ]:
duplicates = df_train.pivot_table(index = ['Data_No'], aggfunc ='size')
df_train_total_sets = duplicates.count()
df_train_total_sets

Continue the numbering in the next set : **df_test**

In [ ]:
new_data_no = df_test['Data_No'] + df_train_total_sets
new_data_no

Replace new data references into **df_test**

In [ ]:
df_test['Data_No'] = new_data_no
df_test

#### Convert `Data_No` to a categorical variable

For the **test** set

In [ ]:
data_no_test = df_test['Data_No'].map(str)
df_test['Data_No'] = data_no_test
df_test.info()

For the **train** set

In [ ]:
data_no_train = df_train['Data_No'].map(str)
df_train['Data_No'] = data_no_train
df_train.info()

In [ ]:
df_train

#### Convert `Dust` to floating number
Derived from the business requirements, we know that the **Dust** categorical variable has a floating number equivalent:
* ISO 12103-1, A2 Fine Test Dust = 0.900 g/m<sup>3</sup>
* ISO 12103-1, A3 Medium Test Dust = 1.025 g/m<sup>3</sup>
* ISO 12103-1, A4 Coarse Test Dust = 1.200 g/m<sup>3</sup>

Convert the Train Set

In [ ]:
dust_density_train = [0.900 if n == 'ISO 12103-1, A2 Fine Test Dust' else (1.025 if n == 'ISO 12103-1, A3 Medium Test Dust' else 1.200) for n in df_train['Dust']]
df_train['Dust'] = dust_density_train
df_train

Confirm the `Dust` data type has changed in **df_train**

In [ ]:
df_train['Dust'].dtype

Convert the Test set using a concatenated function

In [ ]:
dust_density_test = [0.900 if n == 'ISO 12103-1, A2 Fine Test Dust' else (1.025 if n == 'ISO 12103-1, A3 Medium Test Dust' else 1.200) for n in df_test['Dust']]
df_test['Dust'] = dust_density_test
df_test

## Add Calculations

### For **df_test** dataset

#### Change in Differential Pressure
Include change in Differential Pressure calculation

In [ ]:
df_test['change_DP'] = df_test['Differential_pressure'].diff().fillna(0)
df_test.loc[363:368]

Replace first instance of `change_DP` with zero value

In [ ]:
df_test['change_DP'][df_test.Data_No != df_test.Data_No.shift(1)] = 0
df_test.loc[363:368]

Check the change in first and not the last values

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(1)].head()

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(-1)].head()

In [ ]:
# df_test['test'] = df_test['Data_No'].shift() != df_test['Data_No']
# df_test.loc[363:368]

In [ ]:
# df_test['test'].loc[366]

In [ ]:
# type(df_test['test'].loc[366])

In [ ]:
# change = []
# if ((df_test['test'] == True).any()):
#     change = df_test[df_test['test'] == True]
# # change.head()
# index_value = change.index
# index_value

In [ ]:
# if change.index == df_test.index:
#     df_test['change_DP'] = 0
# df_test.loc[363:368]

In [ ]:
# change = df_test[df_test.Data_No != df_test.Data_No.shift(1)].head()
# change.change_DP = 0
# change

In [ ]:
# df_test['change_DP'][df_test.Data_No != df_test.Data_No.shift(1)] = 0
# df_test.loc[363:368]

In [ ]:
# df_test[df_test.Data_No != df_test.Data_No.shift(1)].head()

In [ ]:
# bool_val = list(map(int, df_test['Data_No'].shift() != df_test['Data_No']))
# df_test['test'] = bool_val
# df_test.loc[363:368]

In [ ]:
# type(df_test['test'].loc[366])

In [ ]:
# df_test['change_DP'] = df_test['Differential_pressure'].diff().fillna(0)
# df_test.loc[363:368]

In [ ]:
# if df_test['test'] == True:
#     df_test['change_DP'] = 0
# else:
#     df_test['change_DP'] == df_test['Differential_pressure'].diff().fillna(0)
# df_test[363:368]

In [ ]:
# # if df_test['Data_No'].shift() != df_test['Data_No']:
# if df_test[(df_test.test == 'True')].item():
#     df_test['test'] = 0
# else:
#     df_test['test'] = df_test['Differential_pressure'].diff().fillna(0)

#### Add Mass Calculation

Mass per observation

In [ ]:
# df_test['Mass_g'] = (df_test.Dust_feed/1000)*df_test.Dust
df_test.loc[:,('Mass_g')] = (df_test.Dust_feed/1000)*df_test.Dust
df_test.loc[363:368]

Cumulative Mass

In [ ]:
data = df_test.Data_No
df_test['Cumulative_Mass_g'] = df_test['Mass_g'].groupby(data).cumsum()
df_test.loc[363:368]

#### Represent the total time of the test set at each row

Retrieve the total time for each test

In [ ]:
time_total = df_test['Time'].groupby(data).max().to_frame()
time_total.index.name = None
time_total['Data_No'] = time_total.index
time_total.head()

Map the total time to each observation and place it in the dataset

In [ ]:
total_test_time = df_test['Data_No'].map(time_total.set_index('Data_No')['Time'])
df_test['Tt'] = total_test_time
df_test.loc[363:368]

### Filter Balance %
Calculation to represent the balance to 600Pa `differential_pressure`. At the last value of the dataset, it indicates the amount of **right censoring** has ocurred to each data bin.

In [ ]:
test_data = df_test['Differential_pressure']
df_censor_test = (((600 - test_data)/600)*100).round(decimals = 2)
df_censor_test.loc[363:368]

In [ ]:
df_test['Filter_Balance'] = df_censor_test
df_test.loc[363:368]

Review the last values of each data bin

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(-1)].head()

Extract information on a particular data bin

In [ ]:
df_bin = df_test[df_test['Data_No'] == '52']
df_bin.describe().round(decimals=2)

---

### Repeat calculations for **df_train** dataset

In [ ]:
df_train.loc[446:451]

Change in Differential Pressure

In [ ]:
df_train['change_DP'] = df_train['Differential_pressure'].diff().fillna(0)
df_train

Mass per observation

In [ ]:
# df_train['Mass_g'] = (df_train.Dust_feed/1000)*df_train.Dust
df_train.loc[:,('Mass_g')] = (df_train.Dust_feed/1000)*df_train.Dust
df_train.loc[446:451]

Cumulative Mass

In [ ]:
data = df_train.Data_No
df_train['Cumulative_Mass_g'] = df_train['Mass_g'].groupby(data).cumsum()
df_train.loc[446:451]

#### Total time of the test at each row

Retrieve the total time for each test

In [ ]:
time_total_train = df_train['Time'].groupby(data).max().to_frame()
time_total_train.index.name = None
time_total_train['Data_No'] = time_total_train.index
time_total_train.head()

Map the total time to each observation and place it in the dataset

In [ ]:
total_test_time = df_train['Data_No'].map(time_total_train.set_index('Data_No')['Time'])
df_train['Tt'] = total_test_time
df_train.loc[446:451]

Filter Balance

In [ ]:
train_data = df_train['Differential_pressure']
df_censor_train = (((600 - train_data)/600)*100).round(decimals = 2)
df_train['Filter_Balance'] = df_censor_train
df_train.loc[446:451]

Review the last values of each data bin

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(-1)].head()

In [ ]:
df_test.describe().round(decimals=2)

View the description of cental tendency of the data with a `Data_No` value of `1`

In [ ]:
df_bin = df_train[df_train['Data_No'] == '1']
df_bin.describe().round(decimals=2)
# df_bin

---

## Visualisations

`pip install matplotlib`

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# df_bin.plot(column=['Differential_pressure'], by='Data_No', figsize=(8, 6))
df_bin.plot(kind='line', x='Data_No', y='Differential_pressure', ylim=(0,600), figsize=(8,6), title='Rate of Differential Pressure change in Data_No by Bin No.\n', xlabel='Data_No', ylabel='Differential_pressure\n')
plt.show()

---

### RUL Calculation
As discussed at the [splitting datasets](https://github.com/roeszler/filter-maintenance-predictor/blob/main/README.md#test-train-validation-data) section of the readme document: 
* The Remaining Useful Life variable has been supplied with live data in the **df_test** dataset and not recorded for the training dataset. 
* Notwithstanding, **RUL is a calculated measure** and may prove useful as an additional observation in the final validation stages. 
* Calculating the RUL also aids to highlight the correlation to the `Differential Pressure` and `Time` variables, where:

<p style="text-align: center; font-size: 1rem;">Remaining Useful Life (RUL) = Total time (cycles) to failure for each life test (T) - current time (t)</p>

To test the function, here we will compare the **actual RUL** values supplied to the **calculated RUL** values.

In [ ]:
df_test.loc[1209:1214]

Retrieve the last RUL value of each dataset

In [ ]:
data = df_test.Data_No
RUL_end = df_test['RUL'].groupby(data).min().to_frame()
RUL_end.index.name = None
RUL_end['Data_No'] = RUL_end.index
RUL_end.head()

Rearrange and Drop unnecessary columns into a new dataset for comparison

In [ ]:
RUL_drop = df_test.drop(['Differential_pressure', 'Flow_rate', 'Dust', 'Dust_feed', 'Mass_g', 'Cumulative_Mass_g', 'Filter_Balance'], axis=1)
RUL_compared = RUL_drop[['Data_No',	'Time',	'Tt', 'RUL']]
RUL_compared

Calculate `RUL_Test` and its difference to `RUL` (Actual) to confirm the calculation is accurate

In [ ]:
RUL_Start = RUL_compared['Data_No'].map(RUL_end.set_index('Data_No')['RUL'])
# RUL_compared['RUL_Test'] = (RUL_compared['Tt'] - RUL_compared['Time']) + RUL_Start
# RUL_compared['RUL_Diff'] = round(RUL_compared['RUL'] - RUL_compared['RUL_Test'])
RUL_compared.loc[:,('RUL_Test')] = (RUL_compared.loc[:,('Tt')] - RUL_compared.loc[:,('Time')]) + RUL_Start
RUL_compared.loc[:,('RUL_Diff')] = round(RUL_compared.loc[:,('RUL')] - RUL_compared.loc[:,('RUL_Test')])
RUL_compared.loc[1210:1220]

The RUL calculation is working as we predicted and can be confident to use this in our calculations. 

**An important note**:  
* This calculation **is not predicting the RUL**, merely representing it with the data provided via the observation `Time` and Total Test Time `Tt`. 
* Both time observations are dependant on `Differential_pressure` reaching 600 Pa (i.e. point of filter failure). 
* This condition is not met in this data, so RUL cannot be calculated **until** we have an accurate prediction of when `Differential_pressure` will reach **600 Pa**.

---

## Save Datasets

#### Combine datasets

In [ ]:
# combined_list = [df_test, df_train, df_validate]
# df = pd.concat(combined_list)
# df

#### Save the files to an outputs/../collection folder

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/collection') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

df_train.to_csv(f'outputs/datasets/collection/PredictiveMaintenanceTrain.csv',index=False)
df_test.to_csv(f'outputs/datasets/collection/PredictiveMaintenanceTest.csv',index=False)
# df_validate.to_csv(f'outputs/datasets/collection/PredictiveMaintenanceValidate.csv',index=False)
# df.to_csv(f'outputs/datasets/collection/FilterMaintenancePredictorDataset.csv',index=False)

Now push the changes to your GitHub Repo, using the Git commands (git add, git commit, git push)

---

# Conclusions and Next steps

#### Conclusions: 
* Data supplied without missing observations
* The Data_No references were repeated and corrected

#### Next Steps:
* Data Cleaning

---